In [1]:
from requests import get
from bs4 import BeautifulSoup

In [2]:
def clean_answer(input_text):
    '''function that cleans the answer texts'''
    answer = input_text.strip('\n')
    answer = answer.replace('\n',' ')
    return answer  

In [3]:
def clean_links(link_list):
    '''function that cleans links depending on their target location'''
    clean_links = []
    
    for link in link_list:
        if link == None: # no link in answer
            clean_links.append(None)
            pass
        
        else:
            link = link['href']
            if link[0] == '?': # target is sub page of the faq
                f_link = 'https://www.rug.nl/education/faq/' + link
                clean_links.append(f_link)
            
            elif link[0] == '/': # target is sub page of the rug site
                f_link = 'https://www.rug.nl' + link
                #print(f_link)
                clean_links.append(f_link)
            
            else: # target is a different link
                clean_links.append(link)
                #print(link)

    return clean_links

In [4]:
def retrieve_data(question_list, answer_list, link_list, url, heading_list):
    """function retrieves questions and answers from the specific RUG pages"""
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    data = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
    
    for links in data:
        link = links.findAll('a')
        
        for a in link:
            if a['href'][0] == "?": #check if link is an internal one
                current_url = 'https://www.rug.nl/education/faq/'+a['href']
                response = get(current_url)
                html_soup = BeautifulSoup(response.text, 'html.parser')
                questions = html_soup.find_all(['h2'])
                content = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                s_content = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                
                if len(content) == len(questions): #check structure of current questions and answers
                    for question, answer in zip(questions, content):      
                        if question.text in heading_list: #check if question contains sub questions
                            links = answer.find_all('a')

                            #access the subquestions
                            for a in links:
                                q = a.text
                                
                                if a['href'][0] == "?":
                                    current_url = 'https://www.rug.nl/education/faq/'+a['href']
                                else: 
                                    current_url = a['href']

                                response = get(current_url)
                                html_soup = BeautifulSoup(response.text, 'html.parser')
                                ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

                                #catch errors when external page is formatted differently
                                try:
                                    answer = clean_answer(ans.text)
                                    ans_link = ans.a
                                    link_list.append(ans_link)
                                    answer_list.append(answer)
                                    question_list.append(q)
                                
                                #Structure of external url is different
                                except(AttributeError, KeyError) as error: 
                                    ans = html_soup.find(['p'])
                                    ans_link = ans.a
                                    link_list.append(ans_link)
                                    answer = clean_answer(ans.text)
                                    answer_list.append(answer)
                                    question_list.append(q)
                        else:      
                            question_list.append(question.text)
                            ans_link = answer.a
                            link_list.append(ans_link)
                            answer = clean_answer(answer.text)
                            answer_list.append(answer)   
                            
                else:
                    question = html_soup.find(['h1'])
                    question_list.append(question.text)
                    ans_link = s_content.a
                    link_list.append(ans_link)
                    answer = clean_answer(s_content.text)
                    answer_list.append(answer)
                    
            else:
                question = a.text
                ans_link = links.a
                link_list.append(ans_link)
                answer = clean_answer(links.text)
                question_list.append(question)
                answer_list.append(answer)
        
    return question_list, answer_list, link_list


In [5]:
def write_aiml_file(question_list, answer_list, clean_links):
    '''function that automatically generates aiml file from questions, answers and links in answers'''
    
    with open('aiml_base.aiml', 'w') as f:
        for vraag, antwoord, link in zip(question_list, answer_list, clean_links):

            if link is not None: # there is a link in the answer
                f.write('<category>'+'\n'+ '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
                '<template>'+ '\n'+ antwoord + '\n'+ '<button>' + '\n' + '<text>' + '\n'+ 'Klik hier' 
                + '\n' + '</text>' + '\n' + '<url>'+ '\n' +link+ '\n' + '</url>' + '\n' + '</button>' + '\n' '</template>'+'\n'+'</category>'+'\n'+'\n')

            else:
                f.write('<category>'+'\n'+
              '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
              '<template>'+ '\n'+ antwoord+ '\n'+ '</template>'+'\n'+
              '</category>'+'\n')
    f.close()

In [6]:
#add headings which contain sub_links to this list
heading_list = ["Aanmelding- en selectieprocedure","Onderwijs - honoursprogramma","Bindend Studieadvies (BSA)", 
                "Studeren met een functiebeperking", "Studiedips en andere studieproblemen.", "Studiekeuze", 
                "Studievertraging", "Honours College", "Academische ontwikkeling", "Titulatuur / graden", "Alumni"] 

#currently contains 3 start pages: naast je studie, begeleiding and studieloopbaan 
url_list = ['https://www.rug.nl/education/faq/?tcid=verint_3_7412_7412', 'https://www.rug.nl/education/faq/?tcid=verint_3_7398_7398', 
            'https://www.rug.nl/education/faq/?tcid=verint_3_7399_7399']


In [7]:
question_list = []
answer_list = []
link_list = []

for url in url_list:
    question_list, answer_list, link_list = retrieve_data(question_list, answer_list, link_list, url, heading_list)


In [8]:
clean_links = clean_links(link_list)
write_aiml_file(question_list, answer_list, clean_links)

In [ ]:
#inspect questions and answers
for vraag, antwoord in zip(question_list, answer_list):
    print(vraag,antwoord,'\n')